In [1]:

%cd ".."

/home/diego/PycharmProjects/DSRetreat_Melbourne_DIOLEM


In [2]:

from sklearn.ensemble import  RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
# from scripts.DA_model_extract import extract, extract_old
import sys

from scripts.eo_transport_data import run
sys.path.append('..')
import pandas as pd

def extract(df_train, df_test):

    y_train = df_train['granted']
    y_test = df_test['granted']
    X_train = df_train.drop(['granted', 'id'],axis=1)
    X_test = df_test.drop(['granted', 'id'],axis=1)

    return X_train, y_train, X_test, y_test

def get_random_classifier():
    return RandomForestClassifier(n_estimators=200)


def get_random_regressor():
    return RandomForestRegressor(n_estimators=200)



def get_classifier_xgboost():
    return XGBClassifier(max_depth=2, min_child_weight=2, gamma=0),


def get_best_xgboost():
    return XGBRegressor(max_depth=4, min_child_weight=4, gamma=0.4, subsample=0.8, colsample_bytree=0.7, reg_alpha = 0.5,learning_rate =0.1, n_estimators= 500)

def get_gridsearch_xgboost():
    xgbparams = {
        #'min_child_weight' : [1,2,3,4,5],
        #'max_depth' : [1,2,3,4,5]
        #'gamma': [0.38, 0.39, 0.4, 0.41, 0.42]
        #'subsample': [0.75,  0.775, 0.8, 0.825, 0.85],
        #'colsample_bytree': [0.65, 0.675, 0.7, 0.725, 0.75]
        #'reg_alpha': [0.1, 0.5 , 1, 5, 10]
        #'learning_rate' : [0.1 , 0.001, 0.0001],
        #'n_estimators': [400, 1500, 5000],
 }
    gs = GridSearchCV(XGBRegressor(max_depth=4, min_child_weight=4, gamma=0.4, subsample=0.8, colsample_bytree=0.7, reg_alpha = 0.5, learning_rate =0.1, n_estimators= 500  ), xgbparams,cv =5, scoring='roc_auc', verbose = 10)
    return gs


def print_best_parameters(classifier):
    if hasattr(classifier, "best_estimator_"):
        best_parameters = classifier.best_estimator_.get_params()
        for param_name in sorted(best_parameters.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))


def test_with_classif(classifier, df_train, df_test):
    X_train, y_train, X_test, y_test  = extract(df_train, df_test)
    classifier.fit(X=X_train, y=y_train)
    y_pred = classifier.predict(X_test)
    print_best_parameters(classifier)
    print(roc_auc_score(y_test, y_pred))

def main():

    df_train, df_test = run(base='data', print_info=False)
    xgridboost = get_gridsearch_xgboost()
    test_with_classif(xgridboost , df_train, df_test)
    bestboost = get_best_xgboost()

    test_with_classif(bestboost , df_train, df_test)
    series = pd.Series(bestboost.get_booster().get_fscore())
    print(series.sort_values(ascending=False))

main()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6633 entries, 0 to 6632
Data columns (total 70 columns):
A                               6633 non-null float64
A.                              6633 non-null float64
B                               6633 non-null float64
C                               6633 non-null float64
Dept.No.                        6633 non-null int8
EXTERNAL_ADVISOR                6633 non-null float64
EXT_CHIEF_INVESTIGATOR          6633 non-null float64
Faculty.No.                     6633 non-null int8
Number.of.Successful.Grant      6633 non-null float64
Number.of.Unsuccessful.Grant    6633 non-null float64
RFCD_21                         6633 non-null float64
RFCD_23                         6633 non-null float64
RFCD_24                         6633 non-null float64
RFCD_25                         6633 non-null float64
RFCD_26                         6633 non-null float64
RFCD_27                         6633 non-null float64
RFCD_28                         663

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV] ....................... , score=0.9180684637705537, total=   2.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.6s remaining:    0.0s


[CV] ....................... , score=0.8578065127360902, total=   2.8s
[CV]  ................................................................


...

[CV] ....................... , score=0.8573678356068145, total=   2.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   11.4s remaining:    0.0s


[CV] ....................... , score=0.8220056744041278, total=   2.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   14.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   14.2s finished


	base_score: 0.5
	booster: 'gbtree'
	colsample_bylevel: 1
	colsample_bytree: 0.7
	gamma: 0.4
	learning_rate: 0.1
	max_delta_step: 0
	max_depth: 4
	min_child_weight: 4
	missing: None
	n_estimators: 500
	n_jobs: 1
	nthread: None
	objective: 'reg:linear'
	random_state: 0
	reg_alpha: 0.5
	reg_lambda: 1
	scale_pos_weight: 1
	seed: None
	silent: True
	subsample: 0.8
0.81186299081
0.81186299081
date                            498
sponsor                         353
grant_value                     201
Number.of.Unsuccessful.Grant    189
month                           148
grant_category                  147
Number.of.Successful.Grant      146
A.                              138
Dept.No.                        124
B                               119
A                               116
C                                89
mean_years_in_uni                86
mean_year_of_birth               85
max_year_of_birth                73
RFCD_32                          58
EXT_CHIEF_INVESTIGATOR           